In [1]:
import random
import arcade
import os
from pathlib import Path
from enum import Enum
from PIL import Image

In [17]:
class MobType(): #holds assets common to a group of mobs with same art
    """
    The MobType class holds common assets for a group of identical sprites.  
    The Mob (Mobile OBject) class actually causes the instantiation of the MobType.
    """
    registry ={}

    Action = Enum('Action',['shoot','hurt','slash','spellcast','thrust','walk'])

    actions_base = [Action.walk, Action.hurt]
    actions_full = actions_base + [Action.slash, Action.spellcast, Action.thrust]

    SPRITE_ROOT = Path('.') / 'textures' / 'png'/ '03_final'

    std_sprite_dims = (64,64)
    ov_sprite_dims = (192,192)

 
    sprite_sheet_dims = { #columns, rows of frames in sprite sheets
        Action.shoot:(4,13),
        Action.hurt:(1,6),
        Action.slash:(4,6),
        Action.spellcast:(4,7),
        Action.thrust:(4,8),
        Action.walk:(4,9)
    }
    @classmethod
    def setup_all(cls):
        for mt in cls.registry.values():
            mt.setup()

    def __init__(self, name, actions_w_sizes): #name of sprite sheets
        self.name = name
        self.actions_w_sizes = actions_w_sizes 
        self.textures = {}
        self.img_locns = {a:None for a in MobType.Action} #list of Rects
        MobType.registry[name] = self
        
    def load_textures(self):
        for a in self.actions_w_sizes.keys():
            fn = (MobType.SPRITE_ROOT/f'{a.name}/{self.name}.png').as_posix()
            print(fn, a.name, self.img_locns[a][:5])
            self.textures[a] = arcade.load_textures(fn,self.img_locns[a])


    def setup_img_locns(self): #dict action:(list of boxes (4-tuples) for std, list of boxes for ov)
        def build_sheet_img_locns(s_wd,s_ht,rows,cols):
            return [[s_wd*j,s_ht*i,s_wd,s_ht] for i in range(rows) for j in range(cols)]
        
        for a in  MobType.Action:
            rows,cols = MobType.sprite_sheet_dims[a]
            std_wd,std_ht = MobType.std_sprite_dims
            ov_wd,ov_ht = MobType.ov_sprite_dims
            if self.actions_w_sizes[a] == 0:
                self.img_locns[a] = build_sheet_img_locns(std_wd,std_ht,rows,cols)
            else:
                self.img_locns[a] = build_sheet_img_locns(ov_wd,ov_ht,rows,cols)

    def setup(self):
        self.setup_img_locns()
        self.load_textures()
        
class Mob(arcade.Sprite):

    textures = {}

    def __init__(self, mob_type_name):#mob_type_name should be for an already registered mob type
        super().__init__()
        self.mob_type_name = mob_type_name
        self.mob_type = None
        
        # Track our state
        self.action = MobType.Action.walk
        self.frame = 0
        self.facing = 2 #0 is up; 1 left; 2 down; 3 down
        self.cur_texture = None

    
    def advance_frame(self):
        facing_ct,frame_ct = MobType.sprite_sheet_dims[self.action]
        self.frame = (self.frame + 1) % Mob.frame_ct

    def update_texture(self): #assumes frame and action are accurate
        facing_ct,frame_ct = MobType.sprite_sheet_dims[self.action]
        facing = self.facings if facing_ct==4 else 0 #Hurt has only one facing
        self.cur_texture =  MobType.textures[self.action][facing*frame_ct + self.frame]

    def update_animation(self, delta_time: float = 1 / 60):

        # Figure out if we need to flip face left or right
        if self.change_x < 0 and self.facing == 3:
            self.facing = 1
            self.update_texture()
        elif self.change_x > 0 and self.facing == 1:
            self.facing = 3
            self.update_texture()

        if self.change_x == 0:
            self.action = 'walking'
            self.update_texture()
            return

        self.advance_frame()
        self.update_texture

    def setup(self):
        self.mob_type = MobType.registry[self.mob_type_name]
        


In [18]:
player = Mob('human_light_armor_long_sword')

In [19]:
action_w_sizes = { 
    MobType.Action.shoot:0,
    MobType.Action.hurt:0,
    MobType.Action.slash:1,
    MobType.Action.spellcast:0,
    MobType.Action.thrust:1,
    MobType.Action.walk:0
}

In [20]:
mob_type = MobType('human_light_armor_long_sword',action_w_sizes)

In [21]:
mob_type.setup_img_locns()

In [25]:
mob_type.img_locns[MobType.Action.thrust]

[[0, 0, 192, 192],
 [192, 0, 192, 192],
 [384, 0, 192, 192],
 [576, 0, 192, 192],
 [768, 0, 192, 192],
 [960, 0, 192, 192],
 [1152, 0, 192, 192],
 [1344, 0, 192, 192],
 [0, 192, 192, 192],
 [192, 192, 192, 192],
 [384, 192, 192, 192],
 [576, 192, 192, 192],
 [768, 192, 192, 192],
 [960, 192, 192, 192],
 [1152, 192, 192, 192],
 [1344, 192, 192, 192],
 [0, 384, 192, 192],
 [192, 384, 192, 192],
 [384, 384, 192, 192],
 [576, 384, 192, 192],
 [768, 384, 192, 192],
 [960, 384, 192, 192],
 [1152, 384, 192, 192],
 [1344, 384, 192, 192],
 [0, 576, 192, 192],
 [192, 576, 192, 192],
 [384, 576, 192, 192],
 [576, 576, 192, 192],
 [768, 576, 192, 192],
 [960, 576, 192, 192],
 [1152, 576, 192, 192],
 [1344, 576, 192, 192]]

In [11]:
def test_img_locns(s_wd,s_ht,rows,cols):
            return [[i,j,5,5] for j in range(cols) for i in range(rows)]

In [12]:
test_img_locns(5,7,3,4)

[[0, 0, 5, 5],
 [1, 0, 5, 5],
 [2, 0, 5, 5],
 [0, 1, 5, 5],
 [1, 1, 5, 5],
 [2, 1, 5, 5],
 [0, 2, 5, 5],
 [1, 2, 5, 5],
 [2, 2, 5, 5],
 [0, 3, 5, 5],
 [1, 3, 5, 5],
 [2, 3, 5, 5]]

In [16]:
[[(i,j) for i in range(4)] for j in range(3)]

[[(0, 0), (1, 0), (2, 0), (3, 0)],
 [(0, 1), (1, 1), (2, 1), (3, 1)],
 [(0, 2), (1, 2), (2, 2), (3, 2)]]